In [ ]:
#https://www.kaggle.com/crowdflower/twitter-airline-sentiment/version/4

In [ ]:
!nvidia-smi

Mon Nov 22 17:24:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd drive/MyDrive/notebooks/twitter_senitment_analysis/

/content/drive/MyDrive/notebooks/twitter_senitment_analysis


In [ ]:
!ls

confusion_mrtx_doc2Vec.png   confusion_mrtx_lda.png  topic_10_Data.pkl
confusion_mrtx_ensamble.png  confusion_mrtx_use.png  topic_20_Data.pkl
confusion_mrtx_grdbst.png    main.ipynb		     Tweets.csv


In [ ]:
import pandas as pd
from sklearn import preprocessing


In [ ]:
import pandas as pd
data = pd.read_csv('Tweets.csv')
data.dtypes

tweet_id                          int64
airline_sentiment                object
airline_sentiment_confidence    float64
negativereason                   object
negativereason_confidence       float64
airline                          object
airline_sentiment_gold           object
name                             object
negativereason_gold              object
retweet_count                     int64
text                             object
tweet_coord                      object
tweet_created                    object
tweet_location                   object
user_timezone                    object
dtype: object

In [ ]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.airline_sentiment)
data['categorical_label'] = le.transform(data.airline_sentiment)


In [ ]:
import json
import collections
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
from nltk.tokenize import TweetTokenizer
def text_preprocess(text, tknzr):
    FLAGS = re.MULTILINE | re.DOTALL
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"
    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")
    text = re_sub(r"[A-Za-z]+[@#$%^&*()]+[A-Za-z]*","abuse")
    tokens = tknzr.tokenize(text.lower())
    return tokens #" ".join(tokens)
tknzr=TweetTokenizer(reduce_len=True, preserve_case=False, strip_handles=False)

In [ ]:
X = []
Y = []
for idx in data.index:
  X.append(text_preprocess(data['text'][idx], tknzr))
  Y.append(data['categorical_label'][idx])
# Train-Test splitting
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
labels = ['Negative','Neutral', 'Positive']  # For further use...

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
# This utility function will be used to evaluate the other models also.
def show_performance_data(Y_test, Y_pred, modelname):
  tmp_result = classification_report(Y_test, Y_pred, target_names=labels, output_dict=True)
  print(classification_report(Y_test, Y_pred, target_names=labels))
  cm1 = confusion_matrix(Y_test, Y_pred)
  df_cm = pd.DataFrame(cm1, index = [i for i in labels], columns = [i for i in labels])
  plt.figure(figsize = (7,5))
  sn.heatmap(df_cm, annot=True,cmap='gist_earth_r', fmt='g')
  plt.savefig('confusion_mrtx_'+modelname+'.png',bbox_inches = 'tight')
  return tmp_result

# Word2Vec

In [ ]:
from gensim.models import Word2Vec
# train model
model = Word2Vec(X_train, min_count=1)
# Helper function
def get_w2v_vector(doc):
  tmp = []
  for w in doc:
    try:
      tmp.append(model[w])
    except:
      pass
  return np.mean(tmp, axis=0)
# Convert training text into vectors 
train_vectors_w2v = []
for doc in X_train:
    try:
      train_vectors_w2v.append(get_w2v_vector(doc))
    except Exception:
      print('error...')
# Convert Test text into vectors
test_vectors_w2v = []
for doc in X_test:
    try:
      test_vectors_w2v.append(get_w2v_vector(doc))
    except Exception:
      print('error...')
cosine_similarities = linear_kernel(train_vectors_w2v, test_vectors_w2v)
Y_pred = []
for cs in cosine_similarities.T:
    idx = cs.argsort()[-1]
    Y_pred.append(Y_train[idx])
result_word2vec = show_performance_data(Y_test, Y_pred, 'w2v')

# LDA

In [ ]:
import os
from gensim.corpora import Dictionary
import gensim
from gensim.matutils import cossim
topic = 20
Lda = gensim.models.ldamodel.LdaModel
dictionary = Dictionary(X_train)
train_corpus = [dictionary.doc2bow(doc) for doc in X_train]
model = Lda(corpus=train_corpus, id2word=dictionary, num_topics=topic)
test_corpus = [dictionary.doc2bow(doc) for doc in X_test]
train_x_topics, test_x_topics = [], []
# Convert text data into topic vectors
for t in train_corpus:
  train_x_topics.append(model[t])
for t in test_corpus:
  test_x_topics.append(model[t])
# Prediction
Y_pred = []
for i in range(len(test_x_topics)):
  tst = test_x_topics[i]
  sim = [cossim(tst, tr_t) for tr_t in train_x_topics]
  idx = np.array(sim).argsort()[-1]
  Y_pred.append(Y_train[idx])
result_lda = show_performance_data(Y_test, Y_pred)

# Doc2Vec

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
vec = 100
tagged_corpus = [TaggedDocument(d, [i]) for i, d in enumerate(X_train)]
model = Doc2Vec(tagged_corpus, vector_size=vec, window=3, dm=1, min_count=1, workers=4)
Y_pred = []
for a in X_test:
  test_doc_vector = model.infer_vector(a)
  sims = model.docvecs.most_similar(positive = [test_doc_vector])
  Y_pred.append(Y_train[sims[0][0]])
result_doc2vec = show_performance_data(Y_test, Y_pred, 'doc2vec')

# USE

In [ ]:
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from sklearn.metrics.pairwise import linear_kernel
# Loading USE encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_model = hub.load(module_url)
print ("module %s loaded" % module_url)
def get_use_vector(doc):
  tmp = use_model(doc)
  return np.mean(tmp, axis=0)
# Convert text data into vector
train_vectors = []
for doc in X_train:
    try:
      train_vectors.append(get_use_vector(doc))
    except Exception:
      print('error...')
test_vectors = []
for doc in X_test:
    try:
      test_vectors.append(get_use_vector(doc))
    except Exception:
      print('error...')
Y_pred = []

cosine_similarities = linear_kernel(train_vectors, test_vectors)
for cs in cosine_similarities.T:
    idx = cs.argsort()[-1]
    Y_pred.append(Y_train[idx])
cosine_similarities = linear_kernel(train_vectors, test_vectors)
result_use = show_performance_data(Y_test, Y_pred, 'use')

# KNN using USE embeddings

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(train_vectors, Y_train)
Y_pred = neigh.predict(test_vectors)
result_knn = show_performance_data(Y_test, Y_pred, 'knn')

# Ensamble

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(train_vectors, Y_train)
Y_pred = clf.predict(test_vectors)
result_rnmdfst = show_performance_data(Y_test, Y_pred, 'rndmfst')

#GredientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=15, learning_rate=1.0,max_depth=1, random_state=0).fit(train_vectors, Y_train)
Y_pred = clf.predict(test_vectors)
result_grdbst = show_performance_data(Y_test, Y_pred, 'grdbst')

In [ ]:
N = 3
results = {'Doc2Vec':result_doc2vec, 'Word2Vec':result_word2vec, 'KNN':result_knn, 'USE':result_use, 'Random Forest':result_rnmdfst, 'Gredient Boost':result_grdbst, 'LDA':result_lda}
ind = np.arange(N) 
width = 0.1
p = []
clr = ['b','g','r','c','m','k','y']
i=0
for a,b in results.items():
  tmp = [b['Negative']['precision'], b['Neutral']['precision'], b['Positive']['precision']] # replace 'precision' with 'recall' or 'f1-score' for the next two plots.
  print(tmp)
  p.append(plt.bar(ind+width*i, tmp, width, color = clr[i]))
  i+=1
plt.xlabel("Sentiments")
plt.ylabel('Precision')
plt.title("Precison comparison of all the models with respect to sentiments")
plt.xticks(ind+width,labels)
plt.legend( tuple(p), tuple(results.keys()), loc='upper center', ncol=2)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,6)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
N = 3
ind = np.arange(N) 
width = 0.1
p = []
clr = ['b','g','r','c','m','k','y']
i=0
for a,b in results.items():
  tmp = [b['Negative']['precision'], b['Neutral']['precision'], b['Positive']['precision']]
  print(tmp)
  p.append(plt.bar(ind+width*i, tmp, width, color = clr[i]))
  i+=1
plt.xlabel("Sentiments")
plt.ylabel('Precision')
plt.title("Precison comparison of all the models with respect to sentiments")
plt.xticks(ind+width,labels)
plt.legend( tuple(p), tuple(results.keys()), loc='upper center', ncol=2)
plt.show()
plt.savefig('all_precision.png',bbox_inches = 'tight')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
N = 3
ind = np.arange(N) 
width = 0.1
  
r = []
clr = ['b','g','r','c','m','k','y']
i=0
for a,b in results.items():
  tmp = [b['Negative']['recall'], b['Neutral']['recall'], b['Positive']['recall']]
  print(tmp)
  r.append(plt.bar(ind+width*i, tmp, width, color = clr[i]))
  i+=1

  
plt.xlabel("Sentiments")
plt.ylabel('Recall')
plt.title("Recall comparison of all the models with respect to sentiments")
  
plt.xticks(ind+width,labels)
plt.legend( tuple(r), tuple(results.keys()), loc='upper center', ncol=2)
#plt.show()
plt.savefig('all_recall.png',bbox_inches = 'tight')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
N = 3
ind = np.arange(N) 
width = 0.1
  
f = []
clr = ['b','g','r','c','m','k','y']
i=0
for a,b in results.items():
  tmp = [b['Negative']['f1-score'], b['Neutral']['f1-score'], b['Positive']['f1-score']]
  print(tmp)
  f.append(plt.bar(ind+width*i, tmp, width, color = clr[i]))
  i+=1

  
plt.xlabel("Sentiments")
plt.ylabel('F1-score')
plt.title("F score comparison of all the models with respect to sentiments")
  
plt.xticks(ind+width,labels)
plt.legend( tuple(f), tuple(results.keys()), loc='upper center', ncol=2)
#plt.show()
plt.savefig('all_f_score.png',bbox_inches = 'tight')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
N = 3
ind = np.arange(N) 
width = 0.1
f = []
clr = ['b','g','r','c','m','k','y']
i=0
for a,b in results.items():
  tmp = [b['weighted avg']['precision'], b['weighted avg']['recall'], b['weighted avg']['f1-score']]
  print(tmp)
  f.append(plt.bar(ind+width*i, tmp, width, color = clr[i]))
  i+=1
plt.xlabel("Metrics")
plt.ylabel('score')
plt.title("Over all weighted scores")
  
plt.xticks(ind+width,['Precision', 'Recall', 'F1-score'])
plt.legend( tuple(f), tuple(results.keys()), loc='upper center', ncol=7)
plt.show()
plt.savefig('all_.png',bbox_inches = 'tight')